# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [38]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [37]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

file_path = Path("merged_data.csv")
merged_data = pd.read_csv(file_path)                                 

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [5]:
average_house_values_by_neighbourhood = to_data[["neighbourhood","average_house_value"]].reset_index()
average_house_values_by_neighbourhood.head()

,year,neighbourhood,average_house_value
0,2001,Agincourt North,200388
1,2001,Agincourt South-Malvern West,203047
2,2001,Alderwood,259998
3,2001,Annex,453850
4,2001,Banbury-Don Mills,371864


In [6]:
df_neighbourhood_locations.head()

,neighbourhood,lat,lon
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718


In [7]:
# concat_neighbourhood = pd.concat(
#     [df_neighbourhood_locations, average_house_values_by_neighbourhood],
#     join="inner",
#     axis=1)
# # concat_neighbourhood.reset_index()   
# concat_neighbourhood.head()

### Panel Visualization Functions

In [8]:
# Define Panel visualization functions

def neighbourhood_map():
    
    """Neighbourhood Map"""
    
    # YOUR CODE HERE!
    
    concat_neighbourhood = pd.read_csv('merged_data.csv')
    concat_neighbourhood.sort_values("average_house_value", ascending=False, inplace=True)
    concat_neighbourhood = concat_neighbourhood.drop(columns=["Unnamed: 0"])
    
    return px.scatter_mapbox(
        concat_neighbourhood,
        lat="lat",
        lon="lon",
        size="average_house_value",
        color="average_house_value",
        color_continuous_scale=px.colors.cyclical.IceFire,
        zoom=10,
        title="Data: Avg suburb values Toronto"
    )

In [9]:
def create_bar_chart(data, title, xlabel, ylabel, color):
    """Create a barplot based in the data argument."""
    
    # YOUR CODE HERE!
    
    return data.hvplot.bar(title = title, xlabel = xlabel, ylabel = ylabel, color = color, height =400, rot=60).opts(yformatter="%.0f")
    
def create_line_chart(data, title, xlabel, ylabel, bar_color):
    """Create a line chart based in the data argument."""
    
    # YOUR CODE HERE!
    
    return data.hvplot.line(title = title, xlabel = xlabel, ylabel = ylabel, color = bar_color, height= 400).opts(yformatter="%.0f")

In [10]:
def average_house_value():
    """Average house value for all Toronto's neighbourhoods per year."""
      
    # YOUR CODE HERE!

    average_house_value = to_data.groupby('year')['average_house_value'].mean()
    return create_line_chart(
        average_house_value, 
        "Average House Value", 
        "Year", 
        "Average House Value", 
        "Red")

In [11]:
def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    # YOUR CODE HERE!

    return average_house_values_by_neighbourhood.hvplot.line(
            x='year', 
            groupby='neighbourhood',
            xlabel='Year',
            ylabel='Average House Value',
        title='Data: Average house value by neighbourhood Toronto'
        ).opts(yformatter="%.0f")

In [12]:
def shelter_costs_versus_owned_houses():

    monthly_shelter_costs = to_data[['shelter_costs_owned','shelter_costs_rented']].reset_index()
    monthly_shelter_costs = monthly_shelter_costs.groupby(by=['year']).mean()
    monthly_shelter_costs.head()
    
    return monthly_shelter_costs.hvplot.line(
            x='year', 
            xlabel='Year',
            ylabel='Cost $',
        title='Shelter costs: Owned vs Rented'
        ).opts(yformatter="%.2f")

In [13]:
def number_dwelling_types():
    """Number of dwelling types per year"""
    
    # YOUR CODE HERE!
    
    dwelling_type_neighbourhood = pd.DataFrame(to_data.loc[:,:'other_house'])
    
    return dwelling_type_neighbourhood.hvplot.bar(
        xlabel="Dwelling type",
        ylabel="Number of dwellings",
        height = 400,
        rot=90
    )

In [14]:
def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    # YOUR CODE HERE!

    top_ten_expensive = to_data[['neighbourhood','average_house_value']].reset_index().drop(columns=['year'])
    top_ten_expensive = top_ten_expensive.groupby(['neighbourhood']).mean()
    top_ten_expensive = top_ten_expensive.sort_values(by=['average_house_value'], ascending=False).head(10)
    top_ten_expensive

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html?highlight=reset_index
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html?highlight=drop%20columns example -> df.drop(columns=['B', 'C'])
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html#pandas.core.groupby.GroupBy.mean
# example -> df.groupby('A').mean() -> Groupby one column and return the mean of the remaining columns in each group.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html  example -> df.sort_values(by='col1', ascending=False)
       
    return top_ten_expensive.hvplot.barh(
        xlabel="Neighbourhood",
        ylabel="Average House Value $",
        title = "Data: average house values by neighbourhood",
        rot=45,
        height = 500,
        width = 900
    ).opts( xformatter="%.0f",fontsize={
        'title': 15, 
        'labels': 14, 
        'xticks': 10, 
        'yticks': 10,
    })

In [15]:
def sunburst_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!
    
    return px.sunburst(merged_data, 
        path=["neighbourhood" ,"average_house_value"], 
        values='average_house_value',
        color='average_house_value'
    )

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [16]:
# Create a Title for the Dashboard
dashboard_title = "# Toronto Real Estate Analysis Model"

# Define a welcome text
dashboard_text = "## A model designed to provide Toronto Real Estate market information for investors, use the tabs to see each section."

In [17]:
# Create a tab layout for the dashboard

In [18]:
# Welcome message at the top of the panel in larger font

welcome_column = pn.Column(
    dashboard_title,
    dashboard_text)

In [19]:


front_page_tab = pn.Column("## Local Map of Toronto", neighbourhood_map(), average_house_value())

front_page_tab = pn.Tabs(
    ("Average values per neighbourhood", pn.Column("## Neighbourhood Map by Value", neighbourhood_map())),
    ("Average house values", pn.Column("## Total Toronto average house values by year", average_house_value())))

In [20]:
# Calculate the mean number of dwelling types units per year

dwelling_type_units = to_data.loc[:,'single_detached_house':'other_house']
dwelling_type_units = dwelling_type_units.groupby('year').mean()
dwelling_type_units.head(10)

yearly_market_analysis_tab = pn.Column(pn.Row
            (create_bar_chart(dwelling_type_units.loc[2001], "Dwelling Types in Toronto 2001","2001", "Dwelling Type Units", "green")),
            (create_bar_chart(dwelling_type_units.loc[2006], "Dwelling Types in Toronto 2001","2006", "Dwelling Type Units", "cyan")),
            (create_bar_chart(dwelling_type_units.loc[2011], "Dwelling Types in Toronto 2001","2011", "Dwelling Type Units", "red")),
            (create_bar_chart(dwelling_type_units.loc[2016], "Dwelling Types in Toronto 2001","2016", "Dwelling Type Units", "blue"))
)

In [21]:
shelter_costs_versus_owned_houses_tab = pn.Column("Press the reset button (second from the bottom) on the graph options", shelter_costs_versus_owned_houses())

# Note to Marker, code seems to set up the graph from a y axis range 0 <-> 40,000, so reset button reduces range

In [22]:
neighbourhood_analysis_tab = pn.Column("## Neighbourhood Analysis", average_value_by_neighbourhood(),number_dwelling_types())

neighbourhood_analysis_tab = pn.Tabs(
    ("Neighbourhood average values", pn.Column("## Average value by neighbourhoods", average_value_by_neighbourhood)),
    ("Types of dwellings", pn.Column("## A segmentation of dwelling types", number_dwelling_types())))

# neighbourhood_analysis_tab = pn.Column(
#     #average_value_by_neighbourhood(), 
#     number_dwelling_types(), 
# #     average_house_value_snapshot()
# )

In [23]:
upmarket_neighbourhood_tab = pn.Column("## Upmarket neighbourhoods", top_most_expensive_neighbourhoods(),sunburst_analysis())

upmarket_neighbourhood_tab = pn.Tabs(
    ("Most expensive neighbourhoods", pn.Column("## Top 10 most expensive neighbourhoods", top_most_expensive_neighbourhoods())),
    ("Descending order of suburb value", pn.Column("## Most to least expensive suburbs", sunburst_analysis())))

In [24]:
# Create the main dashboard

dashboard = pn.Column(dashboard_title,
                      dashboard_text,
                      pn.Tabs(
                            ("Overview", front_page_tab),
                            ("Yearly Market Analysis", yearly_market_analysis_tab),
                            ("Shelter Costs versus House Value", shelter_costs_versus_owned_houses_tab),
                            ("Neighbourhood_Analysis", neighbourhood_analysis_tab),
                            ("Up market Luxury Neighbourhoods", upmarket_neighbourhood_tab))
)                      

## Serve the Panel Dashboard

In [25]:
dashboard.servable(dashboard_title)

# Three of the five tabs have sub tabs 

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Tabs
        [0] Tabs
            [0] Column
                [0] Markdown(str)
                [1] Plotly(Figure)
            [1] Column
                [0] Markdown(str)
                [1] HoloViews(Curve)
        [1] Column
            [0] Row
                [0] HoloViews(Bars)
            [1] HoloViews(Bars)
            [2] HoloViews(Bars)
            [3] HoloViews(Bars)
        [2] Column
            [0] Markdown(str)
            [1] HoloViews(NdOverlay)
        [3] Tabs
            [0] Column
                [0] Markdown(str)
                [1] Column
                    [0] Column()
                    [1] Row
                        [0] Row(name='interactive02562')
                            [0] HoloViews(DynamicMap, name='interactive02562')
                            [1] Column
                                [0] WidgetBox
                                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                                [1] VSpacer()
            [1] Column
                [0] Markdown(str)
                [1] HoloViews(Bars)
        [4] Tabs
            [0] Column
                [0] Markdown(str)
                [1] HoloViews(Bars)
            [1] Column
                [0] Markdown(str)
                [1] Plotly(Figure)

In [26]:
# # Debugging

# Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

# However, you can test each plot by uncommenting the following code

In [27]:
# neighbourhood_map().show()

In [28]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [29]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [30]:
# average_house_value()

In [31]:
# average_value_by_neighbourhood()

In [32]:
# number_dwelling_types()

In [33]:
# average_house_value_snapshot()

In [34]:
# top_most_expensive_neighbourhoods()

In [35]:
# sunburts_cost_analysis()


